# SSO + Snowpark Tester

### To sign into snowpark, you need to have login info. but with sos, we have sso so i need to try logging in that way

In [1]:
import numpy as np
import pandas as pd
import json

from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Use pathlib to get the current working directory, 
# then cd.. into the parent folder where the snowpark_credientials.json
# is located.
from pathlib import Path
file_path_split = str(Path.cwd()).split("/")
parent_folder = "/".join(file_path_split)

with open(parent_folder + "/sso_credentials.json", "r") as f:
    connection_parameters = json.load(f)

session = Session.builder.configs(connection_parameters).create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [21]:
# Register this as vectorized function that we can reference in Sigma

def cron_descriptor_to_readable_text_udf(cron_input: str) -> dict:

    from cron_descriptor import get_description
    desc = get_description(cron_input)

    if desc.find(",") == -1:
        desc = desc + ", daily"

    return {'cron_input': cron_input, 'cron_readable': desc}


# Register Vectorized UDF
cron_descriptor_to_readable_text_udf = session.udf.register(func=cron_descriptor_to_readable_text_udf,
                                                        name="CRON_DESCRIPTOR_TO_READABLE_TEXT_UDF",
                                                        stage_location='@MODELS', # can point to any stage in the schema, doesn't mattter which but needs to be included
                                                        input_types=[T.StringType()],
                                                        return_type=T.VariantType(),
                                                        replace=True,
                                                        is_permanent=True,
                                                        packages=['pandas', 'cron-descriptor'],
                                                        max_batch_size=1000,
                                                        session=session)

session.close()

The version of package cron-descriptor in the local environment is 1.2.35, which does not fit the criteria for the requirement cron-descriptor. Your UDF might not work when the package version is different between the server and your local environment
